In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sps
import plotly.graph_objs as go

In [2]:
def get_qq_plot(p_values):
    """Рисует распределение p-value"""
    p_values = np.array(p_values)
    probs = []
    x = [0.01 * i for i in range(101)]
    for i in range(101):
        alpha_step = 0.01 * i
        probs.append(p_values[p_values < alpha_step].shape[0] / p_values.shape[0])
    fig = go.Figure([go.Scatter(x=x, y=probs, mode="markers", name="p_value"),
                 go.Scatter(x=x, y=x, mode="lines", name="uniform")])
    fig.update_layout(height=600, width=600, title="Q-Q plot") 
    return fig

def get_power(p_values, alpha=0.05):
    """Оценка мощности критерия, при условии, что значения p_value взяты при наличии 
    различий в сравниваемых выборках 
    """
    p_values = np.array(p_values)
    return p_values[p_values < alpha].shape[0] / p_values.shape[0] * 100

In [3]:
def duration(k, delta_effect, sigma_1, sigma_2, alpha=0.05, beta=0.2):
    z = sps.norm.ppf(1 - alpha / 2) + sps.norm.ppf(1 - beta)
    n = (k + 1) * z ** 2 * (sigma_1 ** 2 + sigma_2 ** 2 / k) / (delta_effect ** 2)
    return n

In [4]:
def step(dist="norm", test="t", loc=1.0, scale=1.0, p=0.2, effect=0.01, size=100):
    if dist == "norm":
        x_a = sps.norm.rvs(loc=loc, scale=scale, size=size)
        x_b = sps.norm.rvs(loc=loc + effect, scale=scale, size=size)
    elif dist == "bernoulli":
        x_a = sps.bernoulli.rvs(p, size=size)
        x_b = sps.bernoulli.rvs(p+effect, size=size)
    else:
        raise ValueError(f"Distribution type '{dist}' is not supported")
    
    if test == "t":
        p_value = sps.ttest_ind(x_a, x_b, equal_var=False).pvalue
    elif test == "mv":
        p_value = sps.mannwhitneyu(x_a, x_b).pvalue
    else:
        raise ValueError(f"Test type '{test}' is not supported")
        
    return p_value    

def make_experiment(dist="norm", loc=1.0, scale=1.0, p=0.2, effect=0.01, size=100, n_exp=10_000):
        
    p_value_t = np.array(
        [step(dist, "t", loc, scale, p, effect, size) for _ in range(n_exp)]
    )
    p_value_mv = np.array(
        [step(dist, "mv", loc, scale, p, effect, size) for _ in range(n_exp)]
    )
    
    return get_power(p_value_t), get_power(p_value_mv)

def test_duration2(dist="norm", loc=1.0, sigma_1=1.0, sigma_2=1.0, p=0.2, delta_effect=[], alpha=0.05, beta=0.2, n_exp=10_000):

    print(f"Expected criterion's power: {1 - beta}")
    if dist == "norm":
        print(f"Distribution's params: loc = {loc}, scale = {sigma_1}")
    elif dist == "bernoulli":
        print(f"Distribution's params: p = {p}")
        sigma_1 = np.sqrt(p * (1 - p))
        sigma_2 = np.sqrt(p * (1 - p))
    else:
        raise ValueError(f"Distribution type '{dist}' is not supported") 
    
    for de in delta_effect:     
        n = duration(
            k=1,
            delta_effect=de,
            sigma_1=sigma_1,
            sigma_2=sigma_2,
            alpha=0.05
        )
        
        ttest, mv = make_experiment(dist, loc, sigma_1, p, de, int(n // 2), n_exp)
        print(f"Effect: {de:.3} | n_total: {int(n)} | power: ttest = {ttest:.3} mannwhitneyu = {mv:.3}")
    print("------------------------------------------------")

Вариант с нормальным распределением

In [5]:
delta_effect = [3., 4., 5., 7., 10.]
test_duration2(
    dist="norm",
    loc=1,
    sigma_1=100,
    sigma_2=100,
    delta_effect=delta_effect,
    n_exp=1_000
)

Expected criterion's power: 0.8
Distribution's params: loc = 1, scale = 100
Effect: 3.0 | n_total: 34883 | power: ttest = 79.1 mannwhitneyu = 79.7
Effect: 4.0 | n_total: 19622 | power: ttest = 80.7 mannwhitneyu = 79.6
Effect: 5.0 | n_total: 12558 | power: ttest = 80.7 mannwhitneyu = 77.7
Effect: 7.0 | n_total: 6407 | power: ttest = 79.8 mannwhitneyu = 77.3
Effect: 10.0 | n_total: 3139 | power: ttest = 79.8 mannwhitneyu = 77.0
------------------------------------------------


Вариант для отклика

In [6]:
delta_effect = [0.01, 0.015, 0.02, 0.025]
test_duration2(
    dist="bernoulli",
    p=0.2,
    delta_effect=delta_effect,
    n_exp=1_000
)

Expected criterion's power: 0.8
Distribution's params: p = 0.2
Effect: 0.01 | n_total: 50232 | power: ttest = 77.7 mannwhitneyu = 79.8
Effect: 0.015 | n_total: 22325 | power: ttest = 82.3 mannwhitneyu = 79.7
Effect: 0.02 | n_total: 12558 | power: ttest = 79.6 mannwhitneyu = 80.1
Effect: 0.025 | n_total: 8037 | power: ttest = 78.7 mannwhitneyu = 79.0
------------------------------------------------


![pic](./exp_p_2.png)

In [7]:
delta_effect = [0.03, 0.04, 0.05]
test_duration2(
    dist="bernoulli",
    p=0.2,
    delta_effect=delta_effect,
    n_exp=1_000
)

Expected criterion's power: 0.8
Distribution's params: p = 0.2
Effect: 0.03 | n_total: 5581 | power: ttest = 78.0 mannwhitneyu = 77.3
Effect: 0.04 | n_total: 3139 | power: ttest = 76.3 mannwhitneyu = 77.2
Effect: 0.05 | n_total: 2009 | power: ttest = 77.1 mannwhitneyu = 76.8
------------------------------------------------


![pic](./exp_p_5.png)

In [11]:
%%time
x = sps.expon.rvs(loc=100, scale=100, size=1000)
go.Figure([go.Histogram(x=x, nbinsx=100)])

CPU times: user 1.87 ms, sys: 100 µs, total: 1.97 ms
Wall time: 1.9 ms


In [76]:
%%time
def get_lin(res="p", effect=0.):
    n_exp = 1000
    p_values = []
    p_values_lin = []
    for _ in range(n_exp):
        records = []
        for i in range(100):
            n_views = int(sps.expon.rvs(loc=100, scale=100))
            clicks = sps.bernoulli.rvs(p=0.05, size=n_views)
            records.append((n_views, np.sum(clicks), np.sum(clicks) / n_views, "A"))
        for i in range(100):
            n_views = int(sps.expon.rvs(loc=100, scale=100))
            clicks = sps.bernoulli.rvs(p=0.05+effect, size=n_views)
            records.append((n_views, np.sum(clicks), np.sum(clicks) / n_views, "B"))  
        df_data = pd.DataFrame(records, columns=["views", "clicks", "cr", "group"])
        cr_A = df_data[df_data["group"] == "A"]["clicks"].sum() / df_data[df_data["group"] == "A"]["views"].sum()

        df_data["cr_lin"] = df_data["cr"] - cr_A * df_data["views"]

        x_a = df_data[df_data["group"] == "A"]["cr"]
        x_b = df_data[df_data["group"] == "B"]["cr"]
        p_values.append(sps.ttest_ind(x_a, x_b).pvalue)

        x_a = df_data[df_data["group"] == "A"]["cr_lin"]
        x_b = df_data[df_data["group"] == "B"]["cr_lin"]
        p_values_lin.append(sps.ttest_ind(x_a, x_b).pvalue)

    return p_values if res == "p" else p_values_lin
    

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 10 µs


In [77]:
get_qq_plot(get_lin("p", 0))

In [78]:
get_qq_plot(get_lin("lin", 0))

In [48]:
get_qq_plot(get_lin("lin", 0.005))

In [49]:
for effect in [0.005, 0.007, 0.01]:
    pwr = get_power(get_lin("lin", effect))
    lin_pwr = get_power(get_lin("lin", effect))
    print(f"Effect: {effect}, power: {pwr}, lin power: {lin_pwr}")


Effect: 0.005, power: 53.7, lin power: 55.7
Effect: 0.007, power: 82.6, lin power: 81.3
Effect: 0.01, power: 98.1, lin power: 97.39999999999999


In [84]:
%%time
def cuped_exp(effect, res="p"): 
    n_exp = 1000
    p_values = []
    p_values_cuped = []
    size = 1000

    # Pre_experiment
    pre_exp = sps.norm.rvs(loc=100, scale=20, size=size)

    for _ in range(n_exp):
        df_A = pd.DataFrame()
        df_A["user"] = [f"A_{x:5}" for x in range(size)]
        df_A["pre_exp"] = pre_exp
        df_A["payments"] = sps.norm.rvs(loc=1., scale=0.1, size=size) * df_A["pre_exp"]

        df_B = pd.DataFrame()
        df_B["user"] = [f"B_{x:5}" for x in range(size)]
        df_B["pre_exp"] = pre_exp
        df_B["payments"] = sps.norm.rvs(loc=1.+effect, scale=0.1, size=size) * df_B["pre_exp"]

        p_values.append(sps.ttest_ind(df_A["payments"], df_B["payments"]).pvalue)

        x_a = df_A["pre_exp"]
        x_b = df_B["pre_exp"]
        y_a = df_A["payments"]
        y_b = df_A["payments"]
        theta = np.cov(x_a, y_a)[0, 1] / np.std(x_a) ** 2

        df_A["payments_cuped"] = df_A["payments"] - theta * df_A["pre_exp"]
        df_B["payments_cuped"] = df_B["payments"] - theta * df_B["pre_exp"]

        p_values_cuped.append(sps.ttest_ind(df_A["payments_cuped"], df_B["payments_cuped"]).pvalue)
        
    return p_values if res == "p" else p_values_cuped

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11 µs


In [85]:
get_qq_plot(cuped_exp(0.01, "p"))

In [86]:
get_qq_plot(cuped_exp(0.01, "cuped"))

In [90]:
for effect in [0.005, 0.01, 0.02, 0.03]:
    pwr = get_power(cuped_exp(effect, "p"))
    lin_pwr = get_power(cuped_exp(effect, "cuped"))
    print(f"Effect: {effect}, power: {pwr:.3}, cuped power: {lin_pwr:.3}")

Effect: 0.005, power: 0.1, cuped power: 18.3
Effect: 0.01, power: 1.9, cuped power: 60.4
Effect: 0.02, power: 49.8, cuped power: 98.7
Effect: 0.03, power: 98.7, cuped power: 1e+02
